In [9]:
inp = "1999/10/19"

In [10]:
import re
datefmt = re.compile(r"(\d{4})\/(\d{2})\/(\d{2})")

if datefmt.match(inp):
    print("ok")

ok


In [11]:
import logging
import re
import requests
import pandas as pd

class NYTGrid:
    
    BASE_URL = "https://raw.githubusercontent.com/doshea/nyt_crosswords/master/"
    
    def __init__(self, date):
        datefmt = re.compile(r"(\d{4})\/(\d{2})\/(\d{2})")
        if not datefmt.match(date):
            raise ValueError("date must be in the format YYYY/MM/DD")
        ymd = date.split("/")
        self._year = ymd[0]
        self._month = ymd[1]
        self._day = ymd[2]
        self._raw_data = None
        self.title = None
        self.author = None
        self.answers = None
        self.clues = None
        
        
    def fetch_data(self):
        """
            Retrieve data from GitHub repo.
        """
        try:
            target = f"{self.BASE_URL}{self._year}/{self._month}/{self._day}.json"
            r = requests.get(target)
            r.raise_for_status()
        except requests.exceptions.HTTPError as e:
            logging.error(f"Could not fetch data for date {self._year}/{self._month}/{self._day}. {e}")
            return
        self._raw_data = r.json()
        self.title = self._raw_data["title"]
        self.author = self._raw_data["author"]
        self.size = self._raw_data["size"]
        self.answers = self._raw_data["answers"]
        self.clues = self._raw_data["clues"]
        
    def __len__(self):
        """
            This assumes that all grids are square.
        """
        return self.size["cols"]
    
    def __str__(self):
        out = "{:^50}".format(self.title)
        out += "\n"
        for i in range(self.size["rows"]):
            start = self.size["cols"]*(i)
            end = self.size["cols"]*(i+1)
            out += "\n"
            out+= (" | ".join(self._raw_data["grid"][start:end]).replace(".", " "))
        return out

    def print_question_answers(self, orientation: str = "across"):
        for question, answer in zip(self.clues[orientation], self.answers[orientation]):
            print(f"{question.ljust(50)}: {answer.rjust(20)}")

    def to_df(self):
        """
            Return a pandas df with columns "clue" and "answer"
        """
        clues = self.clues["across"].copy()
        clues.extend(self.clues["down"])
        answers = self.answers["across"].copy()
        answers.extend(self.clues["down"])
        return pd.DataFrame({"clue": clues, "answer": answers})

In [12]:
g = NYTGrid("1996/10/28")

In [13]:
g.fetch_data()

In [14]:
print(g)

NY TIMES, MON, OCT 28, 1996            

W | A | R | M |   | F | O | R | M | A |   | O | D | D | S
A | L | O | E |   | O | N | E | A | L |   | R | I | O | T
S | T | A | T | U | E | O | F | L | I | B | E | R | T | Y
H | O | N | O | R |   |   |   |   | C | L | O | T | H | E
  |   |   | O | G | R | E |   | C | I | A |   |   |   |  
E | S | C |   | E | M | M | A | L | A | Z | A | R | U | S
S | H | A | M |   | S | O | M | E |   | E | R | A | S | E
S | I | R | E | N |   | T | E | A |   | D | O | Z | E | R
E | N | E | R | O |   | I | N | N | S |   | N | E | R | F
N | E | W | C | O | L | O | S | S | U | S |   | S | S | S
  |   |   |   | D | E | N |   | E | M | I | T |   |   |  
O | T | O | O | L | E |   |   |   |   | Z | O | W | I | E
G | R | O | V | E | R | C | L | E | V | E | L | A | N | D
L | A | Z | E |   | E | P | O | X | Y |   | L | A | K | E
E | Y | E | R |   | D | A | N | T | E |   | S | C | A | N


In [15]:
g.print_question_answers(), g.print_question_answers("down")

1. Friendly                                       :                 WARM
5. Pro ___ (perfunctory)                          :                FORMA
10. Vegas calculation                             :                 ODDS
14. Lip balm ingredient                           :                 ALOE
15. Ryan or Tatum                                 :                ONEAL
16. Urban unrest                                  :                 RIOT
17. National monument dedicated 10/28/1886        :      STATUEOFLIBERTY
20. Show respect for                              :                HONOR
21. Dress                                         :               CLOTHE
22. Fairy tale villain                            :                 OGRE
25. Spies' org.                                   :                  CIA
26. PC key                                        :                  ESC
29. 47-Across poet                                :          EMMALAZARUS
35. Farce                                         :

(None, None)

In [16]:
clues = g.clues["across"].copy()
clues.extend(g.clues["down"])
answers = g.answers["across"].copy()
answers.extend(g.clues["down"])
pd.DataFrame({"clue": clues, "answer": answers})

,clue,answer
0,1. Friendly,WARM
1,5. Pro ___ (perfunctory),FORMA
2,10. Vegas calculation,ODDS
3,14. Lip balm ingredient,ALOE
4,15. Ryan or Tatum,ONEAL
...,...,...
73,61. Idyllic place,61. Idyllic place
74,"63. Tax return preparer, for short","63. Tax return preparer, for short"
75,64. Actor Chaney,64. Actor Chaney
76,65. Abbr. after a telephone number,65. Abbr. after a telephone number


{'across': ['1. Friendly',
  '5. Pro ___ (perfunctory)',
  '10. Vegas calculation',
  '14. Lip balm ingredient',
  '15. Ryan or Tatum',
  '16. Urban unrest',
  '17. National monument dedicated 10/28/1886',
  '20. Show respect for',
  '21. Dress',
  '22. Fairy tale villain',
  "25. Spies' org.",
  '26. PC key',
  '29. 47-Across poet',
  '35. Farce',
  '37. "___ Like It Hot"',
  '38. Clear the blackboard',
  '39. Ambulance wail',
  '41. Coffee alternative',
  '42. Catnapper',
  '43. First month of the ano',
  '44. Bed-and-breakfasts',
  "46. Kids' indoor ball material",
  '47. Poem inscribed on 17-Across, with "The"',
  '50. Draft org.',
  '51. Place for thieves',
  '52. Send out',
  '54. Lawrence of Arabia portrayer',
  '58. Cry of delight',
  '62. President who dedicated 17-Across',
  '67. Take it easy',
  '68. Adhesive resin',
  '69. Huron, for one',
  '70. Watcher',
  '71. "The Divine Comedy" poet',
  '72. Examine closely'],
 'down': ['1. Do the dishes',
  '2. Palo ___, Calif.',
  '3